# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Nome: _____

___
### Atenção: <br> $\quad$* Serão permitidos grupos de até 3 pessoas, mas com uma rubrica mais exigente. <br> $\quad$* Veja RUBRICA na última página do arquivo PDF que traz o enunciado do Proejto 1.

___
Carregando algumas bibliotecas:

In [54]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$''<>]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed


In [55]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\danie\OneDrive - Insper - Instituto de Ensino e Pesquisa\CDados\Projeto 1


___

### Carregando a base de dados com as mensagens dos seus arquivos. <br> Tire o `#` do início da linha de código condizente ao caso escolhido para o projeto 1.

In [56]:
nome = input("Digite o username digitado quando arquivo foi criado: ")

In [57]:
# SE SEU PROJETO USA OS DADOS ATÉ TRIO
#train = pd.read_csv('dados_treino_ate_TRIO_'+nome+'.csv')
#test = pd.read_csv('dados_teste_ate_TRIO_'+nome+'.csv')

In [58]:
# SE SEU PROJETO USA OS DADOS SÓ DUPLA"
train = pd.read_csv('dados_treino_so_DUPLA_'+nome+'.csv')
test = pd.read_csv('dados_teste_so_DUPLA_'+nome+'.csv')

In [59]:
train.head()

,Text,target
0,Fi-shock works great. This is number three fo...,1.0
1,Great stuff delivery is very fast . I mix it w...,1.0
2,I purchased these for my mom and she loves the...,1.0
3,I love Eight O'clock coffee but when shopping ...,0.0
4,These are really good. You do need to add jus...,1.0


In [60]:
test.head()

,Text,target
0,I ordered a case of 10 for $[...].00. I love t...,1.0
1,I bought these based on all the great reviews ...,0.0
2,This is a fantastic product. I promote this pr...,0.0
3,A nicely spiced light roast coffee with the ar...,1.0
4,This really is a DELICIOUS product and if you ...,0.0


___
# Classificador Automático (Boot)


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

criar uma base de dados filtrando todos os comentarios com target 1 e outra com target 0


___
## Montando SEU Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
# ===== separa as classes (como você já fazia) =====
um   = train.loc[train.target == 1, "Text"].astype(str)
zero = train.loc[train.target == 0, "Text"].astype(str)

# ===== junta, limpa e tokeniza por classe =====
umstr    = " ".join(um.tolist())
zerostr  = " ".join(zero.tolist())

umlimpo   = cleanup(umstr.lower())
zerolimpo = cleanup(zerostr.lower())

umlista    = umlimpo.split()
zerolista  = zerolimpo.split()
trainlista = umlista + zerolista

# ===== contagens (AGORA EM CONTAGENS, NÃO EM PROPORÇÃO) =====
umseries    = pd.Series(umlista, dtype="object")
zeroseries  = pd.Series(zerolista, dtype="object")
trainseries = pd.Series(trainlista, dtype="object")

# contagens absolutas:
umcount    = umseries.value_counts()      # ex.: 'ótimo' -> 37
zerocount  = zeroseries.value_counts()    # ex.: 'ótimo' -> 5
traincount = trainseries.value_counts()   # ex.: 'ótimo' -> 42

# ===== priors por classe (em LOG) com base no nº de documentos =====
lenum   = len(um)
lenzero = len(zero)
total_docs = lenum + lenzero

probum   = lenum / total_docs
probzero = lenzero / total_docs

log_prior_um   = np.log(probum if probum > 0 else 1e-12)
log_prior_zero = np.log(probzero if probzero > 0 else 1e-12)

# ===== vocabulário e totais de tokens por classe =====
vocab = sorted(set(umcount.index).union(set(zerocount.index)))
V = len(vocab)

total_um_tokens   = int(umcount.sum())
total_zero_tokens = int(zerocount.sum())

# ===== Laplace smoothing (alpha) e log-likelihoods =====
alpha = 1.0

# reindexa para o mesmo vocabulário e aplica smoothing:
umcount_full   = umcount.reindex(vocab, fill_value=0)
zerocount_full = zerocount.reindex(vocab, fill_value=0)

# P(token|classe) com Laplace e em LOG
loglik_um = np.log((umcount_full + alpha) / (total_um_tokens + alpha * V))
loglik_zero = np.log((zerocount_full + alpha) / (total_zero_tokens + alpha * V))

# valor padrão para tokens OOV (não vistos no treino)
log_oov_um   = np.log(alpha / (total_um_tokens + alpha * V))
log_oov_zero = np.log(alpha / (total_zero_tokens + alpha * V))


In [ ]:
def classcome(come: str) -> int:
    """
    Classifica um comentário como 1 (positivo) ou 0 (negativo),
    usando Naive Bayes Multinomial com logs e Laplace smoothing.
    Requer as variáveis calculadas no CODIGO 1.
    """
    if not isinstance(come, str):
        come = "" if come is None else str(come)

    # tokeniza igual ao treino
    comelimpo = cleanup(come.lower())
    comelista = comelimpo.split()

    # começa nos log-priors
    score_um   = log_prior_um
    score_zero = log_prior_zero

    # acumula log P(token|classe) para cada token
    for palavra in comelista:
        # pega loglik; se token não estiver no vocabulário, usa OOV
        score_um   += loglik_um.get(palavra, log_oov_um)
        score_zero += loglik_zero.get(palavra, log_oov_zero)

    # decide pela maior pontuação (maior log-posterior)
    return 1 if score_um > score_zero else 0

In [ ]:
# for i in range(len(test)):
#     classcome(test.iloc[i,0])




: 

___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
## Análise Qualitativa da Performance do Classificador

___
## Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**